In [1]:
import os
import psutil
import geopandas as gpd
from matrixconverters.read_ptv import ReadPTVMatrix
import xarray as xr
from collections import namedtuple, defaultdict

In [2]:
os.chdir("..")

In [3]:
from params.project_params import ENCODING_CP1252, CRS_EPSG_ID_WGS84
from scripts.constants import GEOMETRY
from scripts.functions import get_gdf_mobility_stations, get_gdf_mobility_stations_with_npvm_zone, get_skim

# Read data from files

In [4]:
path_to_npvm_zones_shp = os.path.join('data', 'Verkehrszonen_Schweiz_NPVM_2017_shp.zip')

In [5]:
path_mobility_stations = os.path.join('data', 'mobility-stationen-und-fahrzeuge-schweiz.csv')

In [6]:
FileNameOldNew = namedtuple("FileNameOldNew", ["file_name_old", "file_name_new"])

In [7]:
skim_path_per_type = {
    'PT_JT': FileNameOldNew('DWV_2017_ÖV_Reisezeit_CH_binär.mtx', 'pt_jt.nc'),
    'PT_NT': FileNameOldNew('144_NTR_(OEV).mtx', 'pt_nt.nc'),
    'PT_Dist': FileNameOldNew('DWV_2017_ÖV_Distanz_CH_binär.mtx', 'pt_dist.nc'),
    'Road_JT': FileNameOldNew('DWV_2017_Strasse_Reisezeit_CH_binär.mtx', 'road_jt.nc'),
    'Road_Dist': FileNameOldNew('DWV_2017_Strasse_Distanz_CH_binär.mtx', 'road_dist.nc')
}

In [8]:
process = psutil.Process()

In [9]:
def print_memory_usage():
    print(process.memory_info().rss / 1024 ** 2)

In [10]:
print_memory_usage()

115.93359375


In [11]:
gdf_npvm_zones = gpd.read_file(path_to_npvm_zones_shp, encoding=ENCODING_CP1252).to_crs(CRS_EPSG_ID_WGS84)
print_memory_usage()

204.82421875


In [12]:
gdf_mobility_stations = get_gdf_mobility_stations(path_mobility_stations)
print_memory_usage()

209.13671875


In [13]:
gdf_mobility_stations

,Stationsnummer,Name,Standort,easting,northing,geometry
0,1006,Brugg Bahnhof,"47.48154,8.20942",8.20942,47.48154,POINT (8.20942 47.48154)
1,1012,Arbon Bahnhof,"47.51032,9.43345",9.43345,47.51032,POINT (9.43345 47.51032)
2,1019,Basel Vogesenstrasse,"47.5686935,7.5748328",7.5748328,47.5686935,POINT (7.57483 47.56869)
3,1024,Bellinzona Stazione,"46.1963,9.03017",9.03017,46.1963,POINT (9.03017 46.19630)
4,1025,Brig Bahnhof,"46.31906,7.99028",7.99028,46.31906,POINT (7.99028 46.31906)
...,...,...,...,...,...,...
1545,6007,Adnovum Teststation2,"47.144722,8.434389",8.434389,47.144722,POINT (8.43439 47.14472)
1546,6028,Convadis Teststandort 401115,"47.503635,8.239223",8.239223,47.503635,POINT (8.23922 47.50364)
1547,6029,Convadis Teststandort 400611,"47.50362,8.23921",8.23921,47.50362,POINT (8.23921 47.50362)
1548,6030,Convadis Teststandort 202773,"47.50368,8.23923",8.23923,47.50368,POINT (8.23923 47.50368)


In [17]:
gdf_mobility_stations_with_npvm_zone = get_gdf_mobility_stations_with_npvm_zone(gdf_mobility_stations, gdf_npvm_zones)
print_memory_usage()

214.3359375


In [18]:
gdf_mobility_stations_with_npvm_zone

,Stationsnummer,ID,Name,geometry,easting,northing
0,1006,409501008,Brugg Bahnhof,POINT (8.20942 47.48154),8.20942,47.48154
719,3215,409501008,Brugg Post Neumarkt / Bahnhofstrasse,POINT (8.20757 47.48216),8.20757,47.48216
1,1012,440101010,Arbon Bahnhof,POINT (9.43345 47.51032),9.43345,47.51032
2,1019,270101028,Basel Vogesenstrasse,POINT (7.57483 47.56869),7.5748328,47.5686935
3,1024,500201015,Bellinzona Stazione,POINT (9.03017 46.19630),9.03017,46.1963
...,...,...,...,...,...,...
1539,5989,626601007,Sion Rue de Lausanne,POINT (7.35072 46.23041),7.350719,46.230406
1540,5990,219601017,Fribourg Bourg / Tilleul,POINT (7.16110 46.80615),7.1611,46.80615
1546,6028,404401001,Convadis Teststandort 401115,POINT (8.23922 47.50364),8.239223,47.503635
1547,6029,404401001,Convadis Teststandort 400611,POINT (8.23921 47.50362),8.23921,47.50362


In [19]:
npvm_zones_with_mobility_station = list(set(gdf_mobility_stations_with_npvm_zone['ID'].values))

# Filter skims to mobility station and write files to disk

In [ ]:
def read_skim_filter_and_write(ty, path_old, path_new):
    print_memory_usage()
    print(path_old, path_new)
    skim = ReadPTVMatrix(os.path.join('data', path_old))
    if ty.startswith('PT'):
        skim_filtered = skim.sel(destinations=npvm_zones_with_mobility_station).matrix.rename({"destinations": "zone_mobility_station"})
    elif ty.startswith('Road'):
        skim_filtered = skim.sel(origins=npvm_zones_with_mobility_station).matrix.rename({"origins": "zone_mobility_station"})
    skim_filtered.to_netcdf(os.path.join('data', path_new))
    print_memory_usage()

In [ ]:
skim_per_type = {}

In [ ]:
for ty, (name_old, name_new) in skim_path_per_type.items():
    read_skim_filter_and_write(ty, name_old, name_new)

# Read filtered files from disk

In [ ]:
%%time
print_memory_usage()
skim_filtered_per_type = {}
skims = []
for ty, (_, name_new) in skim_path_per_type.items():
    skim_filtered_per_type[ty] = xr.open_dataset(os.path.join('data', name_new))
print_memory_usage()

# Run query data

## Origin and destination NPVM-zone

In [ ]:
from_zone_id = 223401003
to_zone_id = 352101001

# Filter skims to origin and destination

In [ ]:
def calc_costs_df(df_data, vtts, pt_min_per_transfer, pt_chf_per_km, road_chf_per_km):
    return vtts / 60 * (df_data['pt_jt'] + pt_min_per_transfer * df_data['pt_nt'] + df_data['road_jt']) + pt_chf_per_km * df_data['pt_dist'] + road_chf_per_km * df_data['road_dist']

In [ ]:
def get_relevant_mob_stations(best_mobility_stations_costs_per_vtts):
    relevant_mob_stations = []
    for df_ in best_mobility_stations_costs_per_vtts.values():
        relevant_mob_stations += df_['zone_mobility_station'].to_list()
    relevant_mob_stations = set(relevant_mob_stations)
    return relevant_mob_stations

In [ ]:
def execute(from_zone_id, to_zone_id):
    pt_jt = skim_filtered_per_type['PT_JT'].sel(origins=from_zone_id, drop=True).matrix
    pt_nt = skim_filtered_per_type['PT_NT'].sel(origins=from_zone_id, drop=True).matrix
    pt_dist = skim_filtered_per_type['PT_Dist'].sel(origins=from_zone_id, drop=True).matrix

    road_jt = skim_filtered_per_type['Road_JT'].sel(destinations=to_zone_id, drop=True).matrix
    road_dist = skim_filtered_per_type['Road_Dist'].sel(destinations=to_zone_id, drop=True).matrix
    
    df_pt_jt = pt_jt.to_dataframe().reset_index().rename(columns={'matrix': 'pt_jt'})
    df_pt_nt = pt_nt.to_dataframe().reset_index().rename(columns={'matrix': 'pt_nt'})
    df_pt_dist = pt_dist.to_dataframe().reset_index().rename(columns={'matrix': 'pt_dist'})

    df_road_jt = road_jt.to_dataframe().reset_index().rename(columns={'matrix': 'road_jt'})
    df_road_dist = road_dist.to_dataframe().reset_index().rename(columns={'matrix': 'road_dist'})
    
    df_data = df_pt_jt
    for df in [df_pt_nt, df_pt_dist, df_road_jt, df_road_dist]:
        df_data = df_data.merge(df)
    
    best_mobility_stations_costs_per_vtts = {}
    for vtts in range(0, 205, 5):
        df_data['costs'] = calc_costs_df(df_data, vtts, 20.0, 0.16, 0.75)
        min_costs = df_data['costs'].min()
        best_mobility_stations_costs_per_vtts[vtts] = df_data[['zone_mobility_station', 'costs']][df_data['costs'] <= 1.02 * min_costs]
    
    relevant_mob_stations = get_relevant_mob_stations(best_mobility_stations_costs_per_vtts)
    
    data_per_zone = df_data[df_data.zone_mobility_station.isin(relevant_mob_stations)].to_dict('records')
    data_per_zone = {x['zone_mobility_station']: x for x in data_per_zone}
    
    mob_stations_per_npvm_zone = defaultdict(list)
    infos_per_mob_station = {}
    for e in gdf_mobility_stations_with_npvm_zone[gdf_mobility_stations_with_npvm_zone.ID.isin(relevant_mob_stations)].to_dict('records'):
        mob_st_nr = e['Stationsnummer']
        mob_st_name = e['Name']
        zone_id = e['ID']
        easting = e['easting']
        northing = e['northing']
        mob_stations_per_npvm_zone[zone_id] += [mob_st_nr]
        if mob_st_nr in infos_per_mob_station:
            raise ValueError('something wrong')
        infos_per_mob_station[mob_st_nr] = {
            'station_nr': mob_st_nr,
            'station_name': mob_st_name,
            'zone_id': zone_id,
            'station_easting': easting,
            'station_northing': northing
        }
    return {
        'best_mobility_stations_costs_per_vtts': {k: df.to_dict('records') for k, df in best_mobility_stations_costs_per_vtts.items()}, 
        'data_per_zone': data_per_zone,
        'mobility_stations_per_zone': dict(mob_stations_per_npvm_zone),
        'infos_per_mobility_station': infos_per_mob_station
    }

In [ ]:
%%time
res = execute(from_zone_id, to_zone_id)

In [ ]:
import json
json.dumps(res)